In [ ]:
from hanlp_restful import HanLPClient
import json
import os
import time

HanLP = HanLPClient('https://www.hanlp.com/api', auth='', language='zh')
data=HanLP.parse('中央亞非利加之蠻地探險英國大探險家李秉鐸司徒雷之實地探險談王桐齡巍然聳於沙上者金字塔崢嶸究兀高舉怪異之頭者獅身人面像澎湃洶湧發於古壹之哀音者尼羅河之長流赤帽黑衣悠悠高然據於駝背上者土人之怪狀亞非利加東北一隅之埃及自旅行者觀之已屬一種不可思議之國', 
            tasks='ner/ontonotes')

In [2]:

tags=data['ner/ontonotes']
print(tags)


[[['亞非利加', 'LOC', 1, 2], ['英國', 'NORP', 6, 7], ['李秉鐸', 'PERSON', 9, 10], ['司徒雷', 'PERSON', 10, 11], ['王桐齡', 'PERSON', 15, 16], ['尼羅河', 'LOC', 43, 44], ['亞非利加', 'LOC', 62, 63], ['埃及', 'GPE', 66, 67]]]


In [3]:
def split_into_blocks(text, segment_size=150):
    # split text into a list of segments
    segment_list = []
    i = -1
    while True:
        i += 1
        left, right = i*segment_size, (i+1)*segment_size
        segment = text[left: right]
        segment_list.append(segment)
        if right >= len(text):
            break
    return segment_list

In [4]:
block_size = 150
big_block_size = 14000
results_dict = dict()

In [5]:
def join(ll):
    # merge a list of lists
    return_l = []
    for l in ll:
        return_l = return_l + l
    return return_l

In [6]:
results_dict = dict()

In [7]:
texts = []
query_timestamps = []
root = "all_cre"
for file_name in os.listdir(root):
    print("currently processing", file_name)
    if file_name in results_dict:
        continue
    with open(os.path.join(root, file_name), 'r', encoding='utf-8') as rf:
        text = rf.read()
#         def read_in_blocks(text, block_size=15000):
#             while True:
#                 block = text.read(block_size)
#             if not block:
#                 return  # 文件读取完毕
#         read_in_blocks(text)
    
    all_tokens = []
    all_tags = []
    
    big_block_list = split_into_blocks(text, 14000)
    
    for big_block in big_block_list:
        block_list = split_into_blocks(big_block, 150)

        if (len(query_timestamps) < 60):
            gap = 10000
        else:
            current_time = time.time()
            gap = current_time - query_timestamps[-60]
        if gap < 70:
            print("\tsleeping for", 70-gap)
            time.sleep(70-gap)
        query_timestamps.append(time.time())
        
        # 分词
        data = HanLP.parse(block_list, tasks='ner/ontonotes')
        tokens = data['tok/fine']
        tags = data['ner/ontonotes']

        all_tokens = all_tokens + join(tokens)
        all_tags = all_tags + join(tags)

    results_dict[file_name] = {"all_tokens": all_tokens, "all_tags": all_tags}
#     print (all_tokens)

currently processing 不可思议.txt
currently processing 中央亚非利加之蛮地探险.txt
currently processing 冒险夜行.txt
currently processing 冒险奇谈.txt
currently processing 冒险小说孤店村.txt
currently processing 冒险小说柏麦船长航海谈.txt
currently processing 冒险小说火山岛.txt
currently processing 冒险小说航海家儿.txt
currently processing 冒险小说金沙窟.txt
currently processing 冒险小说鬼域.txt
currently processing 冒险成功之商界伟人.txt
currently processing 冢中怪.txt
currently processing 北冰洋归客谈.txt
currently processing 南斐洲脱险记.txt
currently processing 叶兰.txt
currently processing 司宾洛.txt
currently processing 回风.txt
currently processing 地下都会与海底森林.txt
currently processing 女丈夫.txt
currently processing 小中华.txt
currently processing 岛国归婚记.txt
currently processing 弟.txt
currently processing 怪国探险记.txt
currently processing 探险英雄传.txt
currently processing 新殖民地仲尼島.txt
currently processing 月球殖民地小說.txt
currently processing 树穴书.txt
currently processing 毒洲探险记.txt
currently processing 海上逸史.txt
currently processing 海底奇谈.txt
currently processing 渔父冒险记 冠万.txt
currently processing 渔父冒险

In [8]:
import json

#with open("results_dict_ner_tag.json", "w", encoding='utf-8') as f:
    #json.dump(results_dict, f)



In [ ]:
with open("results_dict_ner_tag.json", "r", encoding='utf-8') as f:
     results_dict = json.load(f)

In [9]:
tags_we_care = {'LOC', 'NORP', 'GPE'}
for file_name in results_dict:
    all_tokens, all_tags = results_dict[file_name]['all_tokens'], results_dict[file_name]['all_tags']
    
    count_dict = dict()
    for [phrase, tag, _, _] in all_tags:
        if tag in tags_we_care:
            key = (phrase, tag)
            if key not in count_dict:
                count_dict[key] = 1
            else:
                count_dict[key] += 1
    results_dict[file_name]['tag_freq'] = count_dict

In [10]:
def merge_dict(dict_list):
    return_dict = dict()
    for d in dict_list:
        # merge d into return_dict
        for key, val in d.items():
            if key not in return_dict:
                return_dict[key] = 0
            return_dict[key] += val
    return return_dict

In [11]:
all_tag_freq = merge_dict([results_dict[file_name]['tag_freq'] for file_name in results_dict])

In [12]:
l = [(key, val) for (key, val) in all_tag_freq.items()]
l.sort(key=lambda var: var[1], reverse=True)
l

[(('中國', 'NORP'), 128),
 (('中國', 'GPE'), 107),
 (('日本', 'GPE'), 55),
 (('古巴', 'GPE'), 54),
 (('上海', 'GPE'), 41),
 (('非洲', 'LOC'), 37),
 (('英國', 'NORP'), 29),
 (('希臘', 'NORP'), 29),
 (('倫敦', 'GPE'), 27),
 (('日本', 'NORP'), 27),
 (('南洋', 'LOC'), 25),
 (('紐約', 'GPE'), 25),
 (('英國', 'GPE'), 24),
 (('漢', 'GPE'), 23),
 (('鄂薄', 'LOC'), 22),
 (('孟買', 'GPE'), 20),
 (('匈奴', 'NORP'), 18),
 (('東京', 'GPE'), 18),
 (('地球', 'LOC'), 17),
 (('中華', 'NORP'), 17),
 (('亞洲', 'LOC'), 17),
 (('香港', 'GPE'), 16),
 (('日本人', 'NORP'), 16),
 (('漢人', 'NORP'), 16),
 (('疏勒', 'GPE'), 15),
 (('美國', 'GPE'), 15),
 (('華人', 'NORP'), 14),
 (('印度', 'GPE'), 13),
 (('歐洲', 'LOC'), 12),
 (('南極', 'LOC'), 12),
 (('蘭箬河', 'LOC'), 12),
 (('泗水', 'LOC'), 11),
 (('大夏', 'LOC'), 11),
 (('龜茲', 'LOC'), 11),
 (('英', 'GPE'), 10),
 (('俄', 'GPE'), 10),
 (('北極', 'LOC'), 10),
 (('西域', 'LOC'), 10),
 (('天津', 'GPE'), 10),
 (('湖南', 'GPE'), 10),
 (('迷魂島', 'LOC'), 10),
 (('聚蠆村', 'LOC'), 10),
 (('北冰洋', 'LOC'), 9),
 (('美洲', 'LOC'), 9),
 (('龜茲', 'GPE'), 9),


In [13]:
import pandas as pd
df = pd.DataFrame([[a,b,c] for ((a, b), c) in l], columns=['phrase', 'tag', 'frequency'])
print(df)

     phrase   tag  frequency
0        中國  NORP        128
1        中國   GPE        107
2        日本   GPE         55
3        古巴   GPE         54
4        上海   GPE         41
...     ...   ...        ...
1038    西格羅   LOC          1
1039    希臘國   GPE          1
1040     江蘇   GPE          1
1041      瑤   LOC          1
1042      沙   LOC          1

[1043 rows x 3 columns]


In [14]:
df.to_excel('all_freq.xlsx', sheet_name='new_sheet_name')

In [ ]:
results_dict['黄金毒.txt']['tag_freq']